# Neutrino ID loader

### Imports

In [1]:
%matplotlib inline
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import uproot
import pickle
import pandas as pd
import numpy as np
import helpers.helpfunction as helper
import gc

<IPython.core.display.Javascript object>

### Constants

Data Run1: 
- data_bnb_run1_C1_high_lifetime (for now, small subset of 5e19) 
    - POT: 2.57e19 (old, 1.922e19)
    - E1DCNT_wcut: 5711101 (old, 4271472)
- data_extbnb_run3_G1_high_lifetime
    - EXT: 15392048 (old, 6200046)
    
Data Run2:
- data_bnb_run2_D2
    - POT: 1.593e+20
    - E1DCNT_wcut: 38186979
- data_extbnb_run2_D2
    - EXT: 14220894
    
Data Run3:
- data_bnb_run3_G1_high_lifetime
    - POT: 3.04e19
    - E1DCNT_wcut: 7301284
- data_extbnb_run3_G1_high_lifetime
    - EXT: 7003049

In [3]:
# pot = [2.57e19, 1.593e20, 3.04e19]
# en1dcnt = [5711101, 38186979, 7301284]
# ext = [15392048, 14220894, 7003049]

<IPython.core.display.Javascript object>

In [4]:
path = "./input/Jan2020/pickled/"

min_lepton_kine = 0.02
min_e = min_lepton_kine + 0.000511
min_mu = min_lepton_kine + 0.105658
pot_per_event = {
    "nu": 1.201e21 / 957702,
    "nue": 5.12e22 / 85774,
    "dirt": 3.08e20 / 98679,
}

run = 1
data_samples = ["on", "off"]
mc_samples = ["nue", "nu", "dirt"]

<IPython.core.display.Javascript object>

In [5]:
# run1_c1 = [4952, 6998]
# run2_d2 = [8406, 11048]
# run3_g1 = [14117, 17566]

run_start = np.loadtxt("input/Jan2020/pot_on_binned.txt", unpack=True)[0]
ext_binned = np.loadtxt("input/Jan2020/pot_off_binned.txt", unpack=True)[3]
pot_binned = np.loadtxt("input/Jan2020/pot_on_binned.txt", unpack=True)[1]
en1dcnt_binned = np.loadtxt("input/Jan2020/pot_on_binned.txt", unpack=True)[2]

mask1 = run_start < 7500
mask2 = (run_start > 7500) & (run_start < 12000)
mask3 = run_start > 12000
masks = [mask1, mask2, mask3]

pot = [sum(pot_binned[m]) for m in masks]
en1dcnt = [sum(en1dcnt_binned[m]) for m in masks]
ext = [sum(ext_binned[m]) for m in masks]

<IPython.core.display.Javascript object>

In [6]:
pot_total = sum(pot_binned)
pot_total

2.2020759999999997e+20

<IPython.core.display.Javascript object>

### Load Samples

#### Load MC

In [31]:
%%time
file_dict = {}
first = True

for sample in mc_samples:
    file = uproot.open("input/Jan2020/run{}/{}_run{}.root".format(run, sample, run))
    file_neutrinoid = file["pandora"]
    file_dict[sample] = {}

    file_dict[sample]["num_events"] = file_neutrinoid["events"].numentries
    file_dict[sample]["pot"] = file_dict[sample]["num_events"] * pot_per_event[sample]
    print(
        sample,
        "num_events:",
        file_dict[sample]["num_events"],
        "pot:",
        file_dict[sample]["pot"],
    )
    file_dict[sample]["events"] = file_neutrinoid["events"].arrays(namedecode="utf-8")
    file_dict[sample]["slices"] = file_neutrinoid["slices"].arrays(namedecode="utf-8")
    file_dict[sample]["flashes"] = file_neutrinoid["flashes"].arrays(namedecode="utf-8")
    file_dict[sample]["showers"] = file["shrreco3d/_rcshr_tree"].arrays(
        namedecode="utf-8"
    )
    file_dict[sample]["metadata"] = file_neutrinoid["metadata"].arrays(
        entrystop=1, namedecode="utf-8", flatten=True
    )
    if first:
        print("\n--- Event Tree ---")
        print(file_dict[sample]["events"].keys())
        print("\n--- Slice Tree ---")
        print(file_dict[sample]["slices"].keys())
        print("\n--- Flash Tree ---")
        print(file_dict[sample]["flashes"].keys())
        print("\n--- Shower Tree ---")
        print(file_dict[sample]["showers"].keys())
        print("\n\n")
        first = False

    # Add the hashes needed to match weights:
    df_to_hash_slice_id = file_neutrinoid["events"].pandas.df(["nuVertex?", "*Energy"])
    slice_id_nu_hash = helper.eventHash(df_to_hash_slice_id)
    file_dict[sample]["events"]["hash"] = slice_id_nu_hash

    # Add a purer selection:
    mask_target = file_dict[sample]["slices"]["isTaggedAsTarget"] == 1
    mask_topo = file_dict[sample]["slices"]["topologicalScore"] > 0.15
    x = file_dict[sample]["slices"]["centerX"]
    y = file_dict[sample]["slices"]["centerY"]
    z = file_dict[sample]["slices"]["centerZ"]
    file_dict[sample]["slices"]["reco_fidvol"] = (helper.is_contain(x, y, z)) & (
        mask_target
    )
    file_dict[sample]["slices"]["pure_selection"] = (mask_topo) & (
        file_dict[sample]["slices"]["reco_fidvol"]
    )
    included_keys = [
        "pure_selection",
        "reco_fidvol",
        "evt_time_sec",
        "evt_time_nsec",
        "event",
    ]
    temp_slice_dict = {
        k: v for k, v in file_dict[sample]["slices"].items() if k in included_keys
    }

    df_temp = pd.DataFrame(temp_slice_dict)
    df_temp["id"] = df_temp["event"].ne(df_temp["event"].shift()).cumsum()
    grouper = df_temp.groupby("id", sort=False)
    no_slices = np.where(file_dict[sample]["events"]["nSlices"] == 0)
    if len(no_slices[0])>0:
        file_dict[sample]["events"]["reco_fidvol"] = np.insert(grouper["reco_fidvol"].max(),no_slices[0],0)
        file_dict[sample]["events"]["pure_selection"] = np.insert(grouper["pure_selection"].max(),no_slices[0],0)
    else:
        file_dict[sample]["events"]["reco_fidvol"] = grouper["reco_fidvol"].max()
        file_dict[sample]["events"]["pure_selection"] = grouper["pure_selection"].max()

nue num_events: 58420 pot: 3.4871919229603376e+22

--- Event Tree ---
dict_keys(['run', 'subRun', 'event', 'evt_time_sec', 'evt_time_nsec', 'nFlashes', 'nFlashesInBeamWindow', 'hasBeamFlash', 'nSlices', 'nSlicesAfterPrecuts', 'foundATargetSlice', 'targetSliceMethod', 'bestCosmicMatch', 'cosmicMatchHypothesis', 'bestCosmicMatchRatio', 'nuMode', 'nuX', 'nuW', 'nuPt', 'nuTheta', 'nuCCNC', 'nuEnergy', 'leptonEnergy', 'nuInteractionTime', 'nuPdgCode', 'nuVertexX', 'nuVertexY', 'nuVertexZ'])

--- Slice Tree ---
dict_keys(['sliceId', 'run', 'subRun', 'event', 'evt_time_sec', 'evt_time_nsec', 'hasDeposition', 'totalCharge', 'centerX', 'centerY', 'centerZ', 'minCRTdist', 'CRTtime', 'CRTplane', 'CRTtracklength', 'CRTnumtracks', 'deltaY', 'deltaZ', 'deltaYSigma', 'deltaZSigma', 'chargeToLightRatio', 'xclVariable', 'passesPreCuts', 'flashMatchScore', 'totalPEHypothesis', 'peHypothesisSpectrum', 'isTaggedAsTarget', 'targetMethod', 'isConsideredByFlashId', 'topologicalScore', 'hasBestTopologicalScor

<IPython.core.display.Javascript object>

In [33]:
for s in mc_samples:
    print(file_dict[s]["num_events"])
    print(len(file_dict[s]["events"]["pure_selection"]))

58420
58420
58876
58876
36930
36930


<IPython.core.display.Javascript object>

In [ ]:
%%time
# Collect the newest weights for the MC samples:
file = pickle.load(
    open(
        "/home/wouter/Documents/Jupyter/searchingfornues/input/16Jan/run1_slimmed.pckl",
        "rb",
    )
)

for sample in mc_samples:
    keys = ["true_nu_vtx_x", "true_nu_vtx_y", "true_nu_vtx_z", "nu_e", "lep_e"]
    this_weight_dict = {}
    for key in keys:
        this_weight_dict[key] = file[sample]["mc"][key]
    weight_hash = helper.eventHash(pd.DataFrame(this_weight_dict))
    weigths = file[sample]["mc"]["weightSplineTimesTune"]
    mapper = dict(zip(weight_hash, weigths))
    
    new_weights = file_dict[sample]["events"]["hash"].map(mapper)
    print(new_weights.describe())
    print(sample)
    file_dict[sample]["events"]["weight"] = (
        file_dict[sample]["events"]["hash"].map(mapper).fillna(1)
    ).values
    
    file_dict[sample]["flashes"]["weight"] = np.repeat(
        file_dict[sample]["events"]["weight"], file_dict[sample]["events"]["nFlashes"]
    )
    file_dict[sample]["slices"]["weight"] = np.repeat(
        file_dict[sample]["events"]["weight"], file_dict[sample]["events"]['nSlices']
    )
    
del file
gc.collect()

In [9]:
run = 1

<IPython.core.display.Javascript object>

In [10]:
%%time
# Define signal categories:
for sample in mc_samples:
    x = file_dict[sample]["events"]["nuVertexX"]
    y = file_dict[sample]["events"]["nuVertexY"]
    z = file_dict[sample]["events"]["nuVertexZ"]
    file_dict[sample]["events"]['true_fidvol'] = helper.is_fid(x, y, z)
    
    

    file_dict[sample]["events"]["nueccinc"] = (
        (file_dict[sample]["events"]["leptonEnergy"] > min_e)
        & (abs(file_dict[sample]["events"]["nuPdgCode"]) == 12)
        & (file_dict[sample]["events"]["nuCCNC"]==0)
        & file_dict[sample]["events"]['true_fidvol']
    )
    file_dict[sample]["events"]["numuccinc"] = (
        (file_dict[sample]["events"]["leptonEnergy"] > min_mu)
        & (abs(file_dict[sample]["events"]["nuPdgCode"]) == 14)
        & (file_dict[sample]["events"]["nuCCNC"]==0)
        & file_dict[sample]["events"]['true_fidvol']
    )
    file_dict[sample]["flashes"]["nueccinc"] = np.repeat(
        file_dict[sample]["events"]["nueccinc"], file_dict[sample]["events"]["nFlashes"]
    )
    file_dict[sample]["slices"]["nueccinc"] = np.repeat(
        file_dict[sample]["events"]["nueccinc"], file_dict[sample]["events"]['nSlices']
    )
    file_dict[sample]["flashes"]["numuccinc"] = np.repeat(
        file_dict[sample]["events"]["numuccinc"], file_dict[sample]["events"]["nFlashes"]
    )
    file_dict[sample]["slices"]["numuccinc"] = np.repeat(
        file_dict[sample]["events"]["numuccinc"], file_dict[sample]["events"]['nSlices']
    )
    file_dict[sample]["flashes"]["true_fidvol"] = np.repeat(
        file_dict[sample]["events"]["true_fidvol"], file_dict[sample]["events"]['nFlashes']
    )
    file_dict[sample]["slices"]["true_fidvol"] = np.repeat(
        file_dict[sample]["events"]["true_fidvol"], file_dict[sample]["events"]['nSlices']
    )

CPU times: user 26.8 ms, sys: 3.89 ms, total: 30.7 ms
Wall time: 30.7 ms


<IPython.core.display.Javascript object>

In [11]:
out_file = open(path + "mc_run{}.pckl".format(run), "wb")
pickle.dump(file_dict, out_file)
out_file.close()

<IPython.core.display.Javascript object>

####  Load Data

In [55]:
run = 3

<IPython.core.display.Javascript object>

In [56]:
file_dict = {}
first = True

for sample in data_samples:
    file = uproot.open("input/Jan2020/run{}/{}_run{}.root".format(run, sample, run))
    file_neutrinoid = file["pandora"]
    file_dict[sample] = {}

    file_dict[sample]["num_events"] = file_neutrinoid["events"].numentries
    print(sample, "num_events:", file_dict[sample]["num_events"])
    file_dict[sample]["events"] = file_neutrinoid["events"].arrays(namedecode="utf-8")
    file_dict[sample]["slices"] = file_neutrinoid["slices"].arrays(namedecode="utf-8")
    file_dict[sample]["flashes"] = file_neutrinoid["flashes"].arrays(namedecode="utf-8")
    file_dict[sample]["showers"] = file["shrreco3d/_rcshr_tree"].arrays(
        namedecode="utf-8"
    )
    file_dict[sample]["metadata"] = file_neutrinoid["metadata"].arrays(
        entrystop=1, namedecode="utf-8", flatten=True
    )

    # Add a purer selection:
    mask_target = file_dict[sample]["slices"]["isTaggedAsTarget"] == 1
    mask_topo = file_dict[sample]["slices"]["topologicalScore"] > 0.15
    x = file_dict[sample]["slices"]["centerX"]
    y = file_dict[sample]["slices"]["centerY"]
    z = file_dict[sample]["slices"]["centerZ"]
    file_dict[sample]["slices"]["reco_fidvol"] = (helper.is_contain(x, y, z) == 1) & (
        mask_target
    )
    file_dict[sample]["slices"]["pure_selection"] = (mask_topo) & (
        file_dict[sample]["slices"]["reco_fidvol"]
    )

    included_keys = [
        "pure_selection",
        "reco_fidvol",
        "evt_time_sec",
        "evt_time_nsec",
        "event",
    ]
    temp_slice_dict = {
        k: v for k, v in file_dict[sample]["slices"].items() if k in included_keys
    }

    df_temp = pd.DataFrame(temp_slice_dict)
    df_temp["id"] = df_temp["event"].ne(df_temp["event"].shift()).cumsum()
    grouper = df_temp.groupby("id", sort=False)

    no_slices = np.where(file_dict[sample]["events"]["nSlices"] == 0)
    if len(no_slices[0]) > 0:
        file_dict[sample]["events"]["reco_fidvol"] = np.insert(
            grouper["reco_fidvol"].max().values, no_slices[0], 0
        )
        file_dict[sample]["events"]["pure_selection"] = np.insert(
            grouper["pure_selection"].max().values, no_slices[0], 0
        )
    else:
        file_dict[sample]["events"]["reco_fidvol"] = grouper["reco_fidvol"].max()
        file_dict[sample]["events"]["pure_selection"] = grouper["pure_selection"].max()

    if first:
        print("\n--- Event Tree ---")
        print(file_dict[sample]["events"].keys())
        print("\n--- Slice Tree ---")
        print(file_dict[sample]["slices"].keys())
        print("\n--- Flash Tree ---")
        print(file_dict[sample]["flashes"].keys())
        print("\n--- Shower Tree ---")
        print(file_dict[sample]["showers"].keys())
        print("\n\n")
        first = False

on num_events: 135486

--- Event Tree ---
dict_keys(['run', 'subRun', 'event', 'evt_time_sec', 'evt_time_nsec', 'nFlashes', 'nFlashesInBeamWindow', 'hasBeamFlash', 'nSlices', 'nSlicesAfterPrecuts', 'foundATargetSlice', 'targetSliceMethod', 'bestCosmicMatch', 'cosmicMatchHypothesis', 'bestCosmicMatchRatio', 'reco_fidvol', 'pure_selection'])

--- Slice Tree ---
dict_keys(['sliceId', 'run', 'subRun', 'event', 'evt_time_sec', 'evt_time_nsec', 'hasDeposition', 'totalCharge', 'centerX', 'centerY', 'centerZ', 'minCRTdist', 'CRTtime', 'CRTplane', 'CRTtracklength', 'CRTnumtracks', 'deltaY', 'deltaZ', 'deltaYSigma', 'deltaZSigma', 'chargeToLightRatio', 'xclVariable', 'passesPreCuts', 'flashMatchScore', 'totalPEHypothesis', 'peHypothesisSpectrum', 'isTaggedAsTarget', 'targetMethod', 'isConsideredByFlashId', 'topologicalScore', 'hasBestTopologicalScore', 'hasBestFlashMatchScore', 'nHits', 'maxDeltaLLMCS', 'lengthDeltaLLMCS', 'ct_result_michel_plane0', 'ct_result_michel_plane1', 'ct_result_michel_p

<IPython.core.display.Javascript object>

In [57]:
# Add the POT counting fields for samples:
file_dict["on"]["pot"] = pot[run - 1]
file_dict["on"]["en1dcnt"] = en1dcnt[run - 1]
file_dict["off"]["ext"] = ext[run - 1]

<IPython.core.display.Javascript object>

In [58]:
out_file = open(path + "data_run{}.pckl".format(run), "wb")
pickle.dump(file_dict, out_file)
out_file.close()

<IPython.core.display.Javascript object>

In [59]:
for s in data_samples:
    print(file_dict[s]["num_events"])
    print(len(file_dict[s]["events"]["pure_selection"]))

135486
135486
83859
83859


<IPython.core.display.Javascript object>

### Done